In [1]:
import os, time, pickle, sys, math
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
from pymol import cmd
from sklearn.metrics import pairwise_distances, pairwise_distances_argmin_min
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import fcluster, linkage, single
from scipy.spatial.distance import pdist
import pymesh

### debug anchor generation

In [2]:
prefix = '/data/lishuya/silexon/masif_anchor_service/'

def load_protein_coords(pdbid, chains, removeHs=True):
    if os.path.exists(prefix+'outputPDB/00-fixed_pdbs/fixed_{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-fixed_pdbs/fixed_{}.pdb'.format(pdbid)
        protein = 'fixed_'+pdbid
    elif os.path.exists(prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)
        protein = pdbid
    else:
        print('pdb file not found')
        return None
    cmd.reinitialize()
    cmd.load(filename)
    if removeHs:
        cmd.remove('hydro')
    protein_coords = []
    cmd.iterate_state(-1, "chain "+"+".join([x for x in chains])+" and not het", "protein_coords.append((x,y,z))", space=locals())
    protein_coords = np.array(protein_coords)
    return protein_coords
    
    
def get_masif_coord_normal(pdbid, datatype):
    if datatype == 'PDB':
        mesh = pymesh.load_mesh(prefix+'outputPDB/01-benchmark_surfaces/{}.ply'.format(pdbid))
    elif datatype == 'PDBbind':
        mesh = pymesh.load_mesh(prefix+'/outputPDBbind/01-benchmark_surfaces/{}.ply'.format(pdbid))
    else:
        raise NotImplementedError
    x = mesh.get_vertex_attribute("vertex_x")
    y = mesh.get_vertex_attribute("vertex_y")
    z = mesh.get_vertex_attribute("vertex_z")
    prt_coord = np.hstack([x,y,z])
    mesh.add_attribute("vertex_normal")
    normal = mesh.get_vertex_attribute("vertex_normal")
    return prt_coord, normal

def check_angle(coord, masif, normal):
    cosine_list = np.matmul((coord-masif)[:,None], normal[:,:,None])[:,0,0]
    return cosine_list

def load_pdb(pdbid, datatype):    
    cmd.reinitialize()
    if datatype == 'PDB':
        cmd.load(prefix+'outputPDB/01-benchmark_pdbs/{}.pdb'.format(pdbid))
    elif datatype == 'PDBbind':
        cmd.load(prefix+'outputPDBbind/01-benchmark_pdbs/{}.pdb'.format(pdbid))
    else:
        raise NotImplementedError
    selName = cmd.get_unused_name("sele_")
    cmd.select(selName, "all")
    exposed = []
    cmd.iterate_state(-1, selName, "exposed.append((x,y,z))", space=locals())
    return np.array(exposed)


In [27]:
def load_protein_coords_2(pdbid, chains, removeHs=True):
    if os.path.exists(prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)
        protein = pdbid
    else:
        print('pdb file not found')
        return None
    cmd.reinitialize()
    cmd.load(filename)
    if removeHs:
        cmd.remove('hydro')
    protein_coords = []
    cmd.iterate_state(-1, "chain "+"+".join([x for x in chains])+" and not het", "protein_coords.append((x,y,z))", space=locals())
    protein_coords = np.array(protein_coords)
    return protein_coords

In [28]:
load_protein_coords_2('1a2k', 'ABCDE').shape

(0,)

In [22]:
def getIndexedFaceSet(wrlname):
    with open(wrlname, 'r') as f:
        for line in f:
            if 'geometry IndexedFaceSet {' in line:
                break
        else:
            print("Could not find IndexedFaceSet")
            return None
        assert 'coord Coordinate' in f.readline()
        assert 'point' in f.readline()
        vertices = []
        for line in f:
            if not line.strip(): continue
            line = line.strip()
            if ']' in line:
                break
            if line.endswith(','):
                line = line[:-1]
            vertices.append([float(item) for item in line.split()])
        assert '}' in f.readline()
        assert 'coordIndex' in f.readline()
        
        for line in f:
            if 'normalPerVertex' in line:
                break
        else:
            print("Could not find normalPerVertex")
            return None
        assert 'normal Normal' in f.readline()
        assert 'vector' in f.readline()
         
        normals = []
        for line in f:
            if not line.strip(): continue
            line = line.strip()
            if ']' in line:
                break
            if line.endswith(','):
                line = line[:-1]
            normals.append([float(item) for item in line.split()[:3]])
    return np.array(vertices), np.array(normals)

def get_mesh_vertices(pdbid, chains):
    if os.path.exists(prefix+'outputPDB/00-fixed_pdbs/fixed_{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-fixed_pdbs/fixed_{}.pdb'.format(pdbid)
        protein = 'fixed_'+pdbid
    elif os.path.exists(prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)
        protein = pdbid
    else:
        print('pdb file not found')
        return None
    print('[DEBUG]:', filename)
    pymol.cmd.reinitialize()
    pymol.cmd.load(filename)
    pymol.cmd.hide('everything')
     
#     pymol.cmd.set('surface_quality', -2)
    pymol.cmd.set('solvent_radius', 1.5)
    pymol.cmd.show('surface', 'chain {}'.format("+".join(chains)))
    pymol.cmd.set_view('1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,300,1')
    pymol.cmd.save('../tmp/' + pdbid + '.wrl')
    vertices, normals = getIndexedFaceSet('../tmp/' + pdbid + '.wrl')
#     os.remove('./tmp/' + pdbid + '.wrl')
    return vertices, normals

In [18]:
os.path.exists('./tmp/' + pdbid + '.wrl')

False

In [12]:
os.path.exists('/data/lishuya/silexon/masif_anchor_service/rpc/process/tmp/' + pdbid + '.wrl')

False

In [25]:
def sample_coords(pdbid, datatype, seed=1234, extend=5, pl_thre=2, pr_thre=5, isprint=False):
    # use masif: pl=0, pr=2.5
    
    protein_exposed = load_protein_coords(*pdbid.split('_'))
    if isprint:
#         print('masif', masif_coords.shape)
        print('protein',protein_exposed.shape)
#         print('normal', normal.shape)
    try:
        pymol_coords, pymol_normal = get_mesh_vertices(*pdbid.split('_'))
        if isprint:
            print('pymol vertices and normals', pymol_coords.shape, pymol_normal.shape)
        pymol_index = np.random.choice(np.arange(len(pymol_coords)), 
                                       int(np.ceil(len(pymol_coords)/10)),
                                       replace=False)
        masif_coords, normal = pymol_coords[pymol_index], pymol_normal[pymol_index]
        if isprint:
            print('pymol vertices and normals sampled', masif_coords.shape, normal.shape)
    except:
        masif_coords, normal = get_masif_coord_normal(pdbid, datatype)
        
    np.random.seed(seed)
    step = 2
    randomness = (np.random.rand(3)*2-1)*step 
    coord_bond_l = protein_exposed.min(axis=0) - extend + randomness
    coord_bond_r = protein_exposed.max(axis=0) + extend + randomness

    n_x, n_y, n_z = int((coord_bond_r[0] - coord_bond_l[0])/step), \
                int((coord_bond_r[1] - coord_bond_l[1])/step), int((coord_bond_r[2] - coord_bond_l[2])/step)
    
    x_coords = np.linspace(coord_bond_l[0], coord_bond_r[0], num = n_x)
    y_coords = np.linspace(coord_bond_l[1], coord_bond_r[1], num = n_y)
    z_coords = np.linspace(coord_bond_l[2], coord_bond_r[2], num = n_z)
    mask = np.ones((n_x, n_y, n_z))
    points = np.where(mask)
    initial_coords = np.array([x_coords[points[0]], y_coords[points[1]], z_coords[points[2]]]).transpose()  
        
    if isprint:
        print('number of initial points', initial_coords.shape)
    
    min_arg, min_dist = pairwise_distances_argmin_min(initial_coords, protein_exposed)
    pp_distance_filter = (min_dist >= pl_thre) & (min_dist <= pr_thre)
    if isprint:
        print('pp_distance_filter', pp_distance_filter.shape, pp_distance_filter.sum())
    
    v1_coords = initial_coords[pp_distance_filter]
    if isprint:
        print('v1_coords', v1_coords.shape)
    min_arg, min_dist = pairwise_distances_argmin_min(v1_coords, masif_coords)
    pm_distance_filter = (min_dist <= 2.5) 
    near_masif_idx = min_arg[pm_distance_filter]
    normal_near = normal[near_masif_idx]
    cosine = check_angle(v1_coords[pm_distance_filter], masif_coords[near_masif_idx], normal_near)
    normal_filter = (cosine>0)
    v2_coords = v1_coords[pm_distance_filter][normal_filter]
    
#     v3_coords = v1_coords[(min_dist > 3)]
#     ac_single = AgglomerativeClustering(n_clusters=None, distance_threshold=2.5, affinity='euclidean', linkage='single')
#     ac_single.fit(v3_coords)
#     count_list = []
#     for ni in range(ac_single.n_clusters_):
#         count_list.append(np.sum(ac_single.labels_==ni))
#     count_list = np.array(count_list)
#     select_cluster = np.where(count_list > 40)[0]
#     if isprint:
#         print('max count', np.max(count_list))
#         print('select', select_cluster, count_list[select_cluster])
#     if len(select_cluster) > 0:
#         v4_coords = np.vstack([v3_coords[ac_single.labels_==ni] \
#                                for ni in select_cluster])
#     else:
#         v4_coords = np.zeros((0,3))
    
#     if isprint:
#         print('v2_coords', v2_coords.shape, 
#               'v3_coords', v3_coords.shape,
#               'v4_coords', v4_coords.shape)
        
#     if len(select_cluster) > 0:
#         coords = np.vstack([v2_coords, v4_coords])
#     else:
#         coords = v2_coords
#     if isprint:
#         print('number of surface points', coords.shape)
        
    return v2_coords

In [20]:
pdbid = '1a2k_ABCDE' #1cdk_ABIJ '2x4o_ABCDEF' # '1nhv_AB' '3hvl_AB' '3ftf_ACD' '3maq_APT'
datatype = 'PDB'
repeat = 1

In [26]:
# outdir = prefix+'outputAnchor/{}_aug_{}/'.format(pdbid, repeat)
# if os.path.exists(outdir + 'anchors.npy'):
#     print('anchor already exists!')
#     exit()

t0 = time.time()
np.random.seed(1234)
seed_list = np.random.randint(0, 10000, repeat)

repeat_list = []
for i_rep in range(repeat):
    points = sample_coords(pdbid, datatype, seed=seed_list[i_rep], isprint=True)
    
    ac_complete = AgglomerativeClustering(n_clusters=None, distance_threshold=6, affinity='euclidean', linkage='complete')
    ac_complete.fit(points)
    centers = []
    for ni in range(ac_complete.n_clusters_):
#         if np.sum([ac_complete.labels_==ni]) <= 5:
#             continue
#         print('number of points in anchor', np.sum([ac_complete.labels_==ni]))
        centers.append(np.mean(points[ac_complete.labels_==ni], axis=0))
    repeat_list.append(np.array(centers))

t1 = time.time()
print('anchor generation aug {} shape {} time {} s'.format(repeat, np.array(repeat_list).shape, t1-t0))

# if not os.path.exists(outdir):
#     os.mkdir(outdir)
# np.save(outdir + 'anchors.npy', repeat_list)

protein (6822, 3)
[DEBUG]: /data/lishuya/silexon/masif_anchor_service/outputPDB/00-fixed_pdbs/fixed_1a2k.pdb
Could not find IndexedFaceSet
number of initial points (137808, 3)
pp_distance_filter (137808,) 14246
v1_coords (14246, 3)
anchor generation aug 1 shape (1, 1476, 3) time 24.01396870613098 s


In [589]:
visualize(pdbid[:4], centers, points=None)

In [485]:
vertices, normals = get_mesh_vertices('1f1v', 'AB')
vertices.shape, normals.shape

((168858, 3), (168858, 3))

In [501]:
# visualize(pdbid[:4], None, vertices)

In [215]:
count_list

array([10129])

### CPI

In [31]:

prefix = '/data/lishuya/silexon/masif_anchor_service/'

def load_pdb(pdbid, datatype):    
    cmd.reinitialize()
    if datatype == 'PDB':
        cmd.load(prefix+'outputPDB/01-benchmark_pdbs/{}.pdb'.format(pdbid))
    elif datatype == 'PDBbind':
        cmd.load(prefix+'outputPDBbind/01-benchmark_pdbs/{}.pdb'.format(pdbid))
    else:
        raise NotImplementedError
    selName = cmd.get_unused_name("sele_")
    cmd.select(selName, "all")
    exposed = []
    cmd.iterate_state(-1, selName, "exposed.append((x,y,z))", space=locals())
    return np.array(exposed)

def sample_coords(pdbid, datatype, start_point, seed=1234, step=1, extend=3, pl_thre=2, pr_thre=4, max_points=800, isprint=False):
    if isprint:
        print('start_point', start_point)
    protein_exposed = load_pdb(pdbid, datatype)
    
    coords = np.array([start_point])
    r = 0
    np.random.seed(seed)
    while coords.shape[0] <= max_points:
        if isprint:
            print('round', r, 'start, coords', coords.shape)
        r += 1
        print('round', r, coords.shape)
        randomness =  (np.random.rand(3)*2-1)*step 
        coord_bond_l = coords.min(axis=0) - extend + randomness
        coord_bond_r = coords.max(axis=0) + extend + randomness
        n_x, n_y, n_z = int((coord_bond_r[0] - coord_bond_l[0])/step), \
                        int((coord_bond_r[1] - coord_bond_l[1])/step), int((coord_bond_r[2] - coord_bond_l[2])/step)
        if isprint:
            print('coord_bond_l', coord_bond_l, 'coord_bond_r', coord_bond_r)
            print('number of initial points', n_x * n_y * n_z, (n_x, n_y, n_z))

        x_coords = np.linspace(coord_bond_l[0], coord_bond_r[0], num = n_x)
        y_coords = np.linspace(coord_bond_l[1], coord_bond_r[1], num = n_y)
        z_coords = np.linspace(coord_bond_l[2], coord_bond_r[2], num = n_z)
        mask = np.ones((n_x, n_y, n_z))
        points = np.where(mask)
        
        initial_coords = np.array([x_coords[points[0]], y_coords[points[1]], z_coords[points[2]]]).transpose()
        print('initial_coords', initial_coords.shape)
        pp_distance = pairwise_distances(protein_exposed, initial_coords)
        pp_distance_filter = (pp_distance.min(axis=0) >= pl_thre) & (pp_distance.min(axis=0) <= pr_thre)
        coords = initial_coords[pp_distance_filter, :]
        print('coords', coords.shape)
        if isprint:
            print('number of in-pocket points', 'coords', coords.shape)
        
        if r>10:
            break
    return coords


In [29]:
pdbid = '2mip_ABG'
datatype = 'PDB'
center = [14.303, 43.290, -5.273]
repeat = 1

In [86]:
pdbid = '3tpx_AB'
datatype = 'PDB'
center = [28.199, -44.075, -3.716]
repeat = 1

In [78]:
pdbid = '2mip_ABG'
datatype = 'PDB'
center = [14.303, 43.290, -5.273]
repeat = 1

In [36]:
np.random.seed(1234)
seed_list = np.random.randint(0, 10000, 10)
print(seed_list)
points = sample_coords(pdbid, datatype, center, int(seed_list[2]), pl_thre=2, isprint=True)
ac_single = AgglomerativeClustering(n_clusters=None, distance_threshold=2, affinity='euclidean', linkage='single')
ac_single.fit(points)

dist_list = []
for ni in range(ac_single.n_clusters_):
    point_set = points[ac_single.labels_==ni]
    if len(point_set) >= 10:
        point_dist = pairwise_distances([center], point_set).reshape(-1)
        dist_list.append(np.mean(point_dist[np.argsort(point_dist)[:10]]))
    else:
        dist_list.append(99)
closest_cluster = np.argmin(dist_list)

new_points = points[ac_single.labels_==closest_cluster]
ac_complete = AgglomerativeClustering(n_clusters=None, distance_threshold=4, affinity='euclidean', linkage='complete')
ac_complete.fit(new_points)
centers = []
for ni in range(ac_complete.n_clusters_):
    centers.append(np.mean(new_points[ac_complete.labels_==ni], axis=0))
anchor = np.array(centers)

anchor.shape

[8915 1318 7221 7540  664 6137 6833 8471 9449 7322]
start_point [14.303, 43.29, -5.273]
round 0 start, coords (1, 3)
round 1 (1, 3)
coord_bond_l [11.06881364 40.68061828 -9.20879303] coord_bond_r [17.06881364 46.68061828 -3.20879303]
number of initial points 216 (6, 6, 6)
initial_coords (216, 3)
coords (1, 3)
number of in-pocket points coords (1, 3)
round 1 start, coords (1, 3)
round 2 (1, 3)
coord_bond_l [12.42770587 41.78315689 -9.51514848] coord_bond_r [18.42770587 47.78315689 -3.51514848]
number of initial points 216 (6, 6, 6)
initial_coords (216, 3)
coords (1, 3)
number of in-pocket points coords (1, 3)
round 2 start, coords (1, 3)
round 3 (1, 3)
coord_bond_l [ 11.71711945  41.95125335 -10.39204818] coord_bond_r [17.71711945 47.95125335 -4.39204818]
number of initial points 216 (6, 6, 6)
initial_coords (216, 3)
coords (3, 3)
number of in-pocket points coords (3, 3)
round 3 start, coords (3, 3)
round 4 (3, 3)
coord_bond_l [ 11.07872643  40.9808383  -11.38768492] coord_bond_r [20.67

(130, 3)

In [88]:
outdir = prefix+'outputAnchor/{}_center_{}_aug_{}/'.format(pdbid, '28.199_-44.075_-3.716', 1)
print(outdir)
if not os.path.exists(outdir):
    os.mkdir(outdir)
np.save(outdir+'anchors.npy'.format(pdbid), [anchor])

/data/lishuya/silexon/masif_anchor_service/outputAnchor/3tpx_AB_center_28.199_-44.075_-3.716_aug_1/


In [89]:
os.listdir(outdir)

['anchors.npy', 'atom_feature.pk']

In [80]:
outdir = prefix+'outputAnchor/{}_center_{}_aug_{}/'.format(pdbid, '14.303_43.290_-5.273', 1)
print(outdir)
if not os.path.exists(outdir):
    os.mkdir(outdir)
np.save(outdir+'anchors.npy'.format(pdbid), [anchor])

/data/lishuya/silexon/masif_anchor_service/outputAnchor/2mip_ABG_center_14.303_43.290_-5.273_aug_1/


In [49]:
# def load_masif_coords(pdbid, datatype):
#     path = prefix+'output{}/04a-precomputation_12A/precomputation/'.format(datatype)
#     x = np.load(path+pdbid+'/p1_X.npy')
#     y = np.load(path+pdbid+'/p1_Y.npy')
#     z = np.load(path+pdbid+'/p1_Z.npy')
#     prt_coord = np.vstack([x,y,z]).transpose()
#     return prt_coord

### visualize

In [467]:
def visualize(pdbid, anchors, points):
    cmd.reinitialize()
    prefix = '/data/lishuya/silexon/masif_anchor_service/'
    if os.path.exists(prefix+'outputPDB/00-fixed_pdbs/fixed_{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-fixed_pdbs/fixed_{}.pdb'.format(pdbid)
        pdbid = 'fixed_'+pdbid
    elif os.path.exists(prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)):
        filename = prefix+'outputPDB/00-raw_pdbs/{}.pdb'.format(pdbid)
    else:
        print('pdb file not found')
        return None
    cmd.load(filename)
    
    if points is not None:
        for x,y,z in points:
            cmd.pseudoatom('points', pos=[x,y,z])
        cmd.color('green', 'points')
    
    if anchors is not None:
        for x,y,z in anchors:
            cmd.pseudoatom('anchors', pos=[x,y,z], vdw='0.5')
        cmd.color('yellow', 'anchors')
        
    
    cmd.color('white', pdbid)
    cmd.show('surface', pdbid)
    cmd.set('bg_rgb', 'white')
    cmd.set('transparency', 0.5)
    cmd.show('spheres', 'anchors')
    
    cmd.save('check/visualize_{}.pse'.format(pdbid))

In [455]:
# anchors = np.load('../../outputAnchor/{}_aug_{}/anchors.npy'.format(pdbid, 1))[0]
# print(anchors.shape)
# visualize(pdbid[:4], anchors, points=None)
# visualize('1nhv', centers, points=None)
visualize(pdbid[:4], centers, points=None)

In [456]:
pdbid

'1cdk_ABIJ'

### rerun coach 420

In [558]:
fw = open('rm_results_coach.sh', 'w')
pdbid_list = []
with open('../pdbid_chain_list_COACH420.txt', 'r') as f:
    for line in f.readlines():
        pdbid = line.strip()
        pdbid_list.append(pdbid)
        fw.write('rm -r /data/lishuya/silexon/masif_anchor_service/outputAnchor/{}_aug_{}/\n'.format(pdbid, 1))
        fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/masif/*{}*\n'.format(pdbid[:4]))
        fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/feature_poc/*{}*\n'.format(pdbid[:4]))
fw.close()
len(pdbid_list)

439

In [559]:
! sh rm_results_coach.sh

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a26_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a2k_ABCDE_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a4k_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a4k_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a7x_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a8t_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1afk_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1arc_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1atl_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b8u_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1blc

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1jsv_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1jyl_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k0n_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k1j_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k22_HIL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k3y_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k54_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k7e_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1k7f_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1kaq_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1ucd_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1uf8_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1um0_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1uvt_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1v0y_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1v3s_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1vcu_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1vh3_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1vps_ABCDE_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1wnz_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2hl0_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2hob_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2hxm_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2hzq_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2i4n_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2i56_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2ihz_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2ioa_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2irx_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2ixl_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2j4e

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3d4p_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3dsr_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3du4_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3duw_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3dwr_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3dzl_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3e3s_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3efv_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3ej0_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/3ek5_ABCDEF_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/

### rerun scPDB

In [607]:
fw = open('rm_results_scPDB.sh', 'w')
pdbid_list = []
with open('deepsurf_scPDB_chain.txt', 'r') as f:
    for line in f.readlines():
        pdbid = line.strip()
        pdbid_list.append(pdbid)
        fw.write('rm -r /data/lishuya/silexon/masif_anchor_service/outputAnchor/{}_aug_{}/\n'.format(pdbid, 1))
        fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/masif/*{}*\n'.format(pdbid[:4]))
        fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/feature_poc/*{}*\n'.format(pdbid[:4]))
fw.close()
len(pdbid_list)

9442

In [612]:
! sh rm_results_scPDB.sh

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1a4r_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bl4_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1eii_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1gzu_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1h35_C_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1h48_DEF_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1h7u_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1h8h_ABCDEFG_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1hwz_ABCDEF_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1kp8_ABCDEFGHIJKLMN_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_servi

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2jg3_DEF_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2kwi_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2nmv_ABD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2ogi_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2pdt_B_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2q3o__aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2qcs_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2qds_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2rc3_AD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2v0i_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2v18_AB

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4j6b_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4k6r_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4kg1_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4knz_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4ktk_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4lcg__aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4m83_B_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4nt3_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4oa9_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4oac_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4otw_A_aug_1

## rerun holo4k

In [533]:
fw = open('rm_results_holo.sh', 'w')
pdbid_list = []
with open('pdbid_chain_list_HOLO4k.txt', 'r') as f:
    for line in f.readlines():
        pdbid = line.strip()
        pdbid_list.append(pdbid)
        fw.write('rm -r /data/lishuya/silexon/masif_anchor_service/outputAnchor/{}_aug_{}/\n'.format(pdbid, 1))
        fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/masif/*{}*\n'.format(pdbid[:4]))
        fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/feature_poc/*{}*\n'.format(pdbid[:4]))
fw.close()
len(pdbid_list)

4130

In [534]:
! sh rm_results_holo.sh

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/121p_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/12as_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/13pk_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/16pk_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/17gs_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/182l_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/183l_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/185l_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/186l_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/187l_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/18gs_AB_a

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b15_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b16_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b1c_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b2l_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b2r_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b37_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b37_C_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b38_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b39_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b3a_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1b4e_A_aug

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bky_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bl6_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1blz_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bmd_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bmk_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bn1_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bn3_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bn4_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bnn_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bnq_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1bnt_A_aug_1/

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c3q_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c3v_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c3w_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c3x_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c4u_123_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c4v_123_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c50_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c5i_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c5n_HIL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1c5o_HIL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cl2_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1clu_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cm0_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cm0_B_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cm8_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cmc_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cml_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cmp_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cnq_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cnw_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1cnx_A_aug_

...

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yds_EI_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yec_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yee_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yef_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yeg_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yei_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yej_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yek_HL_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yet_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1ykf_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/1yl

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2fcr_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2fdx_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2fgi_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2flv_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2fvx_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2fx2_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2glr_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2gpa_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2gsa_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2gsp_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2gsq_A_aug_

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2tpr_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2tps_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2trs_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2trt_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2tsy_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2tys_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2uag_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2udp_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2ukd_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2usn_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/2vgc_ABC

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4daa_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4dbv_OPQR_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4dcg_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4dmr_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4er1_E_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4erk_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4fbp_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4fua_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4fx2_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4gch_EFG_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/4gpb_

rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6gsx_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6gsy_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6jdw_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6lpr_AP_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6nse_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6nul_A_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6prc_CHLM_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6q21_ABCD_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6std_ABC_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6tim_AB_aug_1/': 没有那个文件或目录
rm: 无法删除 '/data/lishuya/silexon/masif_anchor_service/outputAnchor/6

In [511]:
# fw = open('rm_results_coach.sh', 'w')
# pdbid_list = ['1yfr_A', '1yfr_B']
# for pdbid in pdbid_list:
#     fw.write('rm -r /data/lishuya/silexon/masif_anchor_service/outputAnchor/{}_aug_{}/\n'.format(pdbid, 1))
#     fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/masif/*{}*\n'.format(pdbid[:4]))
#     fw.write('rm -rf /data/lishuya/silexon/masif_anchor_service/input/progress/feature_poc/*{}*\n'.format(pdbid[:4]))
# fw.close()
# len(pdbid_list)

In [8]:
# check
todo, doing, done = 0, 0, 0
for fname in os.listdir('../../input/progress/feature_poc'):
    if 'todo' in fname:
        todo += 1
    if 'doing' in fname:
        doing += 1
    if 'done' in fname:
        done += 1
todo, doing, done

(26126, 0, 27487)

In [25]:
# check  pocket feature

todo, doing, done = set(), set(), set()
for fname in os.listdir('../../input/progress/feature_poc'):
    if 'todo' in fname:
        todo.add(fname.split('_')[1])
    if 'doing' in fname:
        doing.add(fname.split('_')[1])
    if 'done' in fname:
        done.add(fname.split('_')[1])
len(todo), len(doing), len(done)

(26603, 0, 27688)

In [26]:
len(todo-done)

0

In [28]:
with open('done.txt', 'w') as f:
    for item in done:
        f.write(item+'\n')
    

In [27]:
len(os.listdir('../../outputAnchor/'))

47266

In [62]:
# check masif

todo, doing, done = set(), set(), set()
for fname in os.listdir('../../input/progress/masif'):
    if 'todo' in fname:
        todo.add(fname.split('_')[1])
    if 'doing' in fname:
        doing.add(fname.split('_')[1])
    if 'done' in fname:
        done.add(fname.split('_')[1])
len(todo), len(doing), len(done)

(47, 1, 47)

In [50]:
# for fname in os.listdir('../../input/progress/masif'):
#     if 'todo' in fname:
#         os.system('touch ../../input/progress/masif/' + fname.replace("todo", "done"))

(35264, 24, 12771)

In [20]:
len(todo-done)

23189

In [301]:
# prefix = '/data/lishuya/silexon/masif_anchor_service/'
# tag_feature_poc_done = prefix+'outputAnchor/AAAA_aug_1/am_list.pt'
# tag_feature_cpi_done = prefix+'outputAnchor/AAAA_center_BBB_aug_1/am_list.npy'
# tag_masif_pdb_done = prefix+'outputPDB/04a-precomputation_12A/precomputation/AAAA/p1_input_feat.npy'
# tag_masif_pdbbind_done = prefix+'outputPDBbind/04a-precomputation_12A/precomputation/AAAA/p1_input_feat.npy'
# tag = 'PDB'

# for item in pdbid_list:
#     pdbid = item.split(' ')[0]              

#     if not os.path.exists('../input/progress/masif/{}todo_{}'.format(tag, pdbid)) and \
#     not os.path.exists('../input/progress/masif/{}doing_{}'.format(tag, pdbid)) and \
#     not os.path.exists('../input/progress/masif/{}done_{}'.format(tag, pdbid)) and \
#     not os.path.exists(tag_masif_pdb_done.replace('AAAA', pdbid)):
#         print('new masif task:', pdbid)
#         # os.system('touch ../input/progress/masif/{}todo_{}'.format(tag, pdbid))

# for item in pdbid_list:
#     pdbid = item.split(' ')[0]
#     if not os.path.exists('../../input/progress/feature_poc/{}todo_{}'.format(tag, pdbid)) and \
#     not os.path.exists('../../input/progress/feature_poc/{}doing_{}'.format(tag, pdbid)) and \
#     not os.path.exists('../../input/progress/feature_poc/{}done_{}'.format(tag, pdbid)) and \
#     not os.path.exists(tag_feature_poc_done.replace('AAAA', pdbid)):
#         if os.path.exists(tag_masif_pdb_done.replace('AAAA', pdbid)):
#             print('new poc feature task:', pdbid)
#             # os.system('touch ../input/progress/feature_poc/{}todo_{}'.format(tag, pdbid))
#         elif os.path.exists('../input/progress/masif/{}done_{}'.format(tag, pdbid)):
#             print('skip failed masif:', pdbid)
#             # os.system('touch ../input/progress/feature_poc/{}done_{}'.format(tag, pdbid))
#         else:
#             print('xxxx')

# done = True
# for item in pdbid_list:
#     pdbid = item.split(' ')[0]
#     done_i = (os.path.exists('../input/progress/feature_poc/{}done_{}'.format('PDB', pdbid)) or \
#              os.path.exists(tag_feature_poc_done.replace('AAAA', pdbid)))
#     done *= done_i
#     if not done_i:
#         print(pdbid)
# done

In [556]:
with open('pdbid_chain_list_COACH420.txt') as f:
    check_list = [x.strip() for x in f.readlines()]
len(check_list)

436

In [557]:
prefix = '/data/lishuya/silexon/masif_anchor_service/'
tag_feature_poc_done = prefix+'outputAnchor/AAAA_aug_1/am_list.pt'
tag_feature_cpi_done = prefix+'outputAnchor/AAAA_center_BBB_aug_1/am_list.npy'
tag_masif_pdb_done = prefix+'outputPDB/04a-precomputation_12A/precomputation/AAAA/p1_input_feat.npy'
tag_masif_pdbbind_done = prefix+'outputPDBbind/04a-precomputation_12A/precomputation/AAAA/p1_input_feat.npy'
tag = 'PDB'

for pdbid in check_list:
    if not os.path.exists(tag_masif_pdb_done.replace('AAAA', pdbid)):
        print('no masif:', pdbid)
        # os.system('touch ../input/progress/masif/{}todo_{}'.format(tag, pdbid))

for pdbid in check_list:
    if not os.path.exists(tag_feature_poc_done.replace('AAAA', pdbid)):
        print('no pocket:', pdbid)

no pocket: 1a26_A
no pocket: 1a2k_ABCDE
no pocket: 1a4k_AB
no pocket: 1a4k_HL
no pocket: 1a7x_AB
no pocket: 1a8t_AB
no pocket: 1afk_AB
no pocket: 1arc_A
no pocket: 1atl_AB
no pocket: 1b8u_A
no pocket: 1blc_A
no pocket: 1bnw_A
no pocket: 1bq3_ABCD
no pocket: 1bqo_AB
no pocket: 1br6_A
no pocket: 1bs1_A
no pocket: 1bsv_A
no pocket: 1bxq_A
no pocket: 1bzy_ABCD
no pocket: 1c1h_A
no pocket: 1c3j_A
no pocket: 1c5i_A
no pocket: 1cen_A
no pocket: 1cet_A
no pocket: 1ch8_A
no pocket: 1cim_A
no pocket: 1cqf_ABCDE
no pocket: 1d3p_ABH
no pocket: 1d4p_ABH
no pocket: 1dcp_ABCDEFGH
no pocket: 1ddt_A
no pocket: 1e3v_AB
no pocket: 1e4g_T
no pocket: 1e5q_ABEFGH
no pocket: 1e5q_CD
no pocket: 1efy_A
no pocket: 1ein_ABC
no pocket: 1esw_A
no pocket: 1etr_HL
no pocket: 1ett_HL
no pocket: 1eve_A
no pocket: 1ex8_A
no pocket: 1exa_A
no pocket: 1eyr_AB
no pocket: 1ezq_AB
no pocket: 1f0t_A
no pocket: 1f4e_A
no pocket: 1f4f_AB
no pocket: 1f4g_AB
no pocket: 1f7b_AC
no pocket: 1f8g_ABCD
no pocket: 1fcv_A
no pocket: 1f

In [425]:
# prefix = '/data/lishuya/silexon/masif_anchor_service/'
# tag_feature_poc_done = prefix+'outputAnchor/AAAA_aug_1/am_list.pt'
# tag_feature_cpi_done = prefix+'outputAnchor/AAAA_center_BBB_aug_1/am_list.npy'
# tag_masif_pdb_done = prefix+'outputPDB/04a-precomputation_12A/precomputation/AAAA/p1_input_feat.npy'
# tag_masif_pdbbind_done = prefix+'outputPDBbind/04a-precomputation_12A/precomputation/AAAA/p1_input_feat.npy'
# tag = 'PDB'

# for pdbid in ['1fpx_A', '1m98_AB', '1pmq_A']:
#     if not os.path.exists(tag_masif_pdb_done.replace('AAAA', pdbid)):
#         print('no masif:', pdbid)
#         # os.system('touch ../input/progress/masif/{}todo_{}'.format(tag, pdbid))


In [89]:
# origial version
# def sample_coords(pdbid, datatype, seed=1234, extend=4, pl_thre=0, pr_thre=2.5, isprint=False):
#     # use masif: pl=0, pr=2.5
#     protein_exposed, normal = get_masif_coord_normal(pdbid, datatype)
# #     protein_exposed = load_protein_coords(*pdbid.split('_'))
    
#     if isprint:
#         print('protein',protein_exposed.shape)
#         print('normal', normal.shape)
    
#     np.random.seed(seed)
#     step = 2
#     randomness = (np.random.rand(3)*2-1)*step 
#     coord_bond_l = protein_exposed.min(axis=0) - extend + randomness
#     coord_bond_r = protein_exposed.max(axis=0) + extend + randomness

#     n_x, n_y, n_z = int((coord_bond_r[0] - coord_bond_l[0])/step), \
#                 int((coord_bond_r[1] - coord_bond_l[1])/step), int((coord_bond_r[2] - coord_bond_l[2])/step)
    
#     x_coords = np.linspace(coord_bond_l[0], coord_bond_r[0], num = n_x)
#     y_coords = np.linspace(coord_bond_l[1], coord_bond_r[1], num = n_y)
#     z_coords = np.linspace(coord_bond_l[2], coord_bond_r[2], num = n_z)
#     mask = np.ones((n_x, n_y, n_z))
#     points = np.where(mask)
#     initial_coords = np.array([x_coords[points[0]], y_coords[points[1]], z_coords[points[2]]]).transpose()  
        
#     if isprint:
#         print('number of initial points', initial_coords.shape)
    
#     min_arg, min_dist = pairwise_distances_argmin_min(initial_coords, protein_exposed)
#     pp_distance_filter = (min_dist >= pl_thre) & (min_dist <= pr_thre)
#     if isprint:
#         print('pp_distance_filter', pp_distance_filter.shape, pp_distance_filter.sum())
    
#     near_masif_idx = min_arg[pp_distance_filter]
#     normal_near = normal[near_masif_idx]
#     cosine = check_angle(initial_coords[pp_distance_filter], protein_exposed[near_masif_idx], normal_near)
#     normal_filter = (cosine>0)
#     if isprint:
#         print('normal_filter', normal_filter.shape, normal_filter.sum())

#     coords = initial_coords[pp_distance_filter][normal_filter]
#     if isprint:
#         print('number of surface points', coords.shape)
        
#     return coords